In [ ]:
import pandas as pd
import numpy as np
import ast
from keras.optimizers import Adam
from Script_model.model import create_model
from keras.preprocessing.image import ImageDataGenerator

def load_images_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    images = []
    for i, row in df.iterrows():
        # Convertir la chaîne en tableau numpy
        img_array = np.array(ast.literal_eval(row['image']))
        images.append(img_array)
    return np.array(images)

Chargement du dataset d'images depuis les fichiers csv

In [ ]:
train_lr_images = load_images_from_csv('dataset/train_lr_df.csv')
train_hr_images = load_images_from_csv('dataset/train_hr_df.csv')
val_lr_images = load_images_from_csv('dataset/val_lr_df.csv')
val_hr_images = load_images_from_csv('dataset/val_hr_df.csv')
test_lr_images = load_images_from_csv('dataset/test_lr_df.csv')
test_hr_images = load_images_from_csv('dataset/test_hr_df.csv')

Normalisation des valeurs des pixels pour qu'elles soient de 0 à 1

In [ ]:
train_lr_images = train_lr_images / 255.0
train_hr_images = train_hr_images / 255.0
val_lr_images = val_lr_images / 255.0
val_hr_images = val_hr_images / 255.0
test_lr_images = test_lr_images / 255.0
test_hr_images = test_hr_images / 255.0

Augmentation des données pour améliorer les résultats de l'entrainement

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,          # Rotation aléatoire de l'image de 20°
    width_shift_range=0.2,      # Décalage horizontal
    height_shift_range=0.2,     # Décalage vertical
    shear_range=0.2,            # Transformation de cisaillement
    zoom_range=0.2,             # Zoom aléatoire
    horizontal_flip=True,       # Retourner l'image horizontalement
    fill_mode='nearest'         # Remplir les pixels vides après transformation
)

# Appliquer l'augmentation aux images
datagen.fit(train_lr_images)

Parametres pour le model

In [ ]:
image_shape = (128, 128, 3)
n1, n2, n3 = 64, 32, 3
f1, f2, f3 = (9, 9), (1, 1), (5, 5)

Importation du model depuis /Script_model/modle.py

In [ ]:
model = create_model(image_shape, n1, f1, n2, f2, n3, f3)

Compilation du model, les paramètres peuvent être à revoir

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


Entrainement du model

In [ ]:
model.fit(
    datagen.flow(train_lr_images, train_hr_images, batch_size=32),
    validation_data=(val_lr_images, val_hr_images),
    epochs=50, 
)

Evaluation du model

In [ ]:
test_loss = model.evaluate(test_lr_images, test_hr_images)
print(f"Test Loss: {test_loss}")

Enregistrement du model afin de pouvoir l'utiliser plus facilement, ou si vous utilisez colab et ne voulez pas relancer l'entrainement

In [ ]:
model.save('/content/my_model.h5')

Telechargement du model


In [ ]:
from google.colab import files

files.download('/content/my_model.h5')
